In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

Instead of concatenating the directional encoding features at the sigma layer we concatenate at the 
beginning of later mlp

In [17]:
class ModelNerfV1(nn.Module):
    def __init__(self):
        super().__init__()
        self.L_pos = 10 ## subject to change
        self.L_dir = 4 ## subject to change 
        pos_enc_features = 3+3*2*self.L_pos #tune this parameters 
        dir_enc_features = 3+3*2*self.L_dir 
        in_features = pos_enc_features
        num_neurons = 256
        
        in_features = pos_enc_features
        self.early_mlp = nn.Sequential(nn.Linear(in_features,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                
        )
        in_features = num_neurons + dir_enc_features+pos_enc_features
        self.later_mlp=nn.Sequential(
                nn.Linear(in_features,256),
                nn.ReLU(),
                nn.Linear(256,256),
                nn.ReLU(),
                nn.Linear(256,256),
                nn.ReLU())
        self.sigma_layer = nn.Linear(num_neurons, num_neurons+1)
        self.pre_final_layer = nn.Sequential(nn.Linear(dir_enc_features+num_neurons+pos_enc_features,num_neurons//2),
                                            nn.ReLU())
       ### Yeah think about what can be done to this layer actually!!!
        
        self.final_layer = nn.Sequential(nn.Linear(num_neurons//2,3), nn.Sigmoid())
        
    def  forward(self, ray_samples, view_dirs):
        rays_samples_encoded = [ray_samples]
        for l_pos in range(self.L_pos):
            rays_samples_encoded.append(torch.sin(2 ** l_pos * torch.pi * ray_samples))
            rays_samples_encoded.append(torch.cos(2 ** l_pos * torch.pi * ray_samples))
        
        rays_samples_encoded = torch.cat(rays_samples_encoded, dim=-1)
        
        
        view_dirs = view_dirs / view_dirs.norm(p=2, dim=-1).unsqueeze(-1)
        view_dirs_encoded = [view_dirs]
        for l_dir in range(self.L_dir):
            view_dirs_encoded.append(torch.sin(2 ** l_dir * torch.pi * view_dirs))
            view_dirs_encoded.append(torch.cos(2 ** l_dir * torch.pi * view_dirs))
        view_dirs_encoded = torch.cat(view_dirs_encoded,dim=-1)
        
        outputs = self.early_mlp(rays_samples_encoded)
        print('Data types')
        print(f'type of view_dirs_encoded:{type(view_dirs_encoded)}')
        print(f'type of rays_samples_encoded:{type(rays_samples_encoded)}')
        print(f'type of outputs:{type(outputs)}')
        outputs = self.later_mlp(torch.cat([outputs, view_dirs_encoded,rays_samples_encoded], dim=-1))
        outputs = self.sigma_layer(outputs)
        sigma_is = torch.relu(outputs[:,0])
        outputs = self.pre_final_layer(torch.cat([view_dirs_encoded, outputs[:, 1:],rays_samples_encoded], dim=-1))
        c_is = self.final_layer(outputs)
        
        return {"c_is":c_is,"sigma_is":sigma_is}
        

In [2]:
# check nerfmodelv1
ds_batch = torch.randn(16834,3)
ray_dirs = torch.randn(16834,3)

In [18]:
model1 = ModelNerfV1()

In [19]:
outputs = model1(ray_dirs,ds_batch)

Data types
type of view_dirs_encoded:<class 'torch.Tensor'>
type of rays_samples_encoded:<class 'torch.Tensor'>
type of outputs:<class 'torch.Tensor'>


In [20]:
outputs['c_is']

tensor([[0.5004, 0.4717, 0.5467],
        [0.5252, 0.4875, 0.5379],
        [0.5305, 0.4967, 0.5348],
        ...,
        [0.5074, 0.5076, 0.5117],
        [0.4863, 0.5236, 0.5465],
        [0.5072, 0.5115, 0.5133]], grad_fn=<SigmoidBackward0>)

In [21]:
outputs['sigma_is']

tensor([0.0203, 0.0015, 0.0127,  ..., 0.0146, 0.0071, 0.0224],
       grad_fn=<ReluBackward0>)

Now we actually explore a skip connection and actually concatenate the positions and directions from the beginning


In [ ]:
## Make it more dense just like skip connections in resnet 
## Essentially at one point we would just add instead of concatenating
class VanillaSkipNerfV1(nn.Module):
    def __init__(self):
        super().__init__()
        self.L_pos = 10 ## subject to change
        self.L_dir = 4 ## subject to change 
        pos_enc_features = 3+3*2*self.L_pos #tune this parameters 
        dir_enc_features = 3+3*2*self.L_dir 
        in_features = pos_enc_features + dir_enc_features
        num_neurons = 256
        
        self.early_mlp = nn.Sequential(nn.Linear(in_features,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                                      )
        in_features = pos_enc_features + num_neurons
        
        self.later_mlp = nn.sequential(nn.Linear(in_features,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                                       nn.Linear(256,256),
                                       nn.ReLU(),
                                      )
        self.sigma_layer = nn.Linear(num_neurons, num_neurons+1)
        self.pre_final_layer = nn.Sequential(nn.Linear(dir_enc_features+num_neurons,num_neurons//2),
                                            nn.ReLU())
        self.final_layer = nn.Sequential(nn.Linear(num_neurons//2,3), nn.Sigmoid())
        
     ## Here we would just add the vanilla skip   
    def forward(self,rays_samples, view_dirs):
        
        rays_samples_encoded = [rays_samples]
        skip1 = 
        for l_pos in range(self.L_pos):
            rays_samples_encoded.append(torch.sin(2**l_pos*torch.pi*rays_samples))
            rays_samples_encoded.append(torch.cos(2**l_pos*torch.pi*rays_samples))
        
        ## Probably after going through few blocks I wanna add it
        rays_samples_encoded = torch.cat(rays_samples_encoded, dim = -1)
        
        ## introduce some skip connections here
        #?
        rays_samples_encoded = rays_samples_encoded+ skip1 ## remember both now have different dimensions
        
        
        view_dirs = view_dirs/view_dirs.norm(p=2, dim=-1).unsqueeze(-1)
        view_dirs_encoded = [view_dirs]
        
        for l_dir in range(self.L_dir):
            view_dirs_encoded.append(torch.sin(2**l_dir*torch.pi*view_dirs))
            view_dirs_encoded.append(torch.cos(2**l_dir*torch.pi*view_dirs))
            
        view_dirs_encoded = torch.cat(view_dirs_encoded, dim=-1)
        

In [ ]:
def make_blocks(self, in_channels,intermediate_channels, num_repeat, expansion, is_Bottleneck, stride):
    

In [6]:
## Vanilla skip nerf architecture with 12 layers of MLP 

class vanilla_skip_nerf_12(nn.Module):
    def __init__(self):
        super().__init__()
        self.L_pos = 10
        self.L_dir = 4
        pos_enc_features = 3*2*self.L_pos ##Don't add 3 here
        dir_enc_features = 3*2*self.L_dir ## Don't add 3 here
        in_pos_features1 = pos_enc_features
        in_dir_features2 = dir_enc_features
        num_neurons = 256
        
        # In one mlp_block, we would feed the pos
        # In the other we would  feed the dir enc
        self.mlp_block_1 = nn.Sequential(
            nn.Linear(in_pos_features1,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        self.mlp_block_2 = nn.Sequential(
            nn.Linear(in_dir_features2, 256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        self.mlp_block_3 = nn.Sequential(
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
#         self.sigma_layer =
#         self.pre_final_layer =
#         self.final_layer =
    def forward(self, rays_samples,view_dirs):
        rays_samples_encoded = [rays_samples]
        for l_pos in range(self.L_pos):
            rays_samples_encoded.append(torch.sin(2**l_pos*torch.pi*rays_samples))
            rays_samples_encoded.append(torch.cos(2**l_pos*torch.pi*rays_samples))
        
        rays_samples_encoded = torch.cat(rays_samples_encoded, dim=-1) ##(16K *60)
        print(f'shape of rays_samples_encoded:{rays_samples_encoded.shape}')
        
        
        view_dirs = view_dirs/view_dirs.norm(p=2, dim=-1).unsqueeze(-1)
        view_dirs_encoded = [view_dirs]
        for l_dir in range(self.L_dir):
            view_dirs_encoded.append(torch.sin(2**l_dir*torch.pi*view_dirs))
            view_dirs_encoded.append(torch.cos(2**l_dir*torch.pi*view_dirs))
        
        view_dirs_encoded = torch.cat(view_dirs_encoded, dim=-1) ##(16*24)
        
        
        print(f'shape of view_dirs_encoded:{view_dirs_encoded.shape}')
        
        outputs_mlp_block1 = self.mlp_block_1(rays_samples_encoded)
        print(f'outputs_mlp_block1:{outputs_mlp_block1.shape}')
        outputs_mlp_block2 = self.mlp_block_2(view_dirs_encoded) ## outputs1 + outputs 2
        print(f'outputs_mlp_block2:{outputs_mlp_block2.shape}')
        # Sum the outputs 
        outputs_combined = outputs_mlp_block1+outputs_mlp_block2
        print(f'outputs_combined:{outputs_combined.shape}')
        # Now Concatenate with positional and directional embeddings
        outputs_mlp_block3 = self.mlp_block_3(outputs_combined)
        print(f'outputs_mlp_block3:{outputs_mlp_block3.shape}')
        
        
        

In [7]:
vanilla_skip_nerf_12_model1 = vanilla_skip_nerf_12()

In [9]:
vanilla_skip_nerf_12_model1(ray_dirs,ds_batch)

shape of rays_samples_encoded:torch.Size([16834, 63])
shape of view_dirs_encoded:torch.Size([16834, 27])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16834x63 and 60x256)

So, Now you see why the +3 was given to self.L_pos

In [10]:
class vanilla_skip_nerf_12(nn.Module):
    def __init__(self):
        super().__init__()
        self.L_pos = 10
        self.L_dir = 4
        pos_enc_features = 3+3*2*self.L_pos 
        dir_enc_features = 3+3*2*self.L_dir 
        in_pos_features1 = pos_enc_features
        in_dir_features2 = dir_enc_features
        num_neurons = 256
        
        # In one mlp_block, we would feed the pos
        # In the other we would  feed the dir enc
        self.mlp_block_1 = nn.Sequential(
            nn.Linear(in_pos_features1,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        self.mlp_block_2 = nn.Sequential(
            nn.Linear(in_dir_features2, 256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        self.mlp_block_3 = nn.Sequential(
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
#         self.sigma_layer =
#         self.pre_final_layer =
#         self.final_layer =
    def forward(self, rays_samples,view_dirs):
        rays_samples_encoded = [rays_samples]
        for l_pos in range(self.L_pos):
            rays_samples_encoded.append(torch.sin(2**l_pos*torch.pi*rays_samples))
            rays_samples_encoded.append(torch.cos(2**l_pos*torch.pi*rays_samples))
        
        rays_samples_encoded = torch.cat(rays_samples_encoded, dim=-1) ##(16K *60)
        print(f'shape of rays_samples_encoded:{rays_samples_encoded.shape}')
        
        
        view_dirs = view_dirs/view_dirs.norm(p=2, dim=-1).unsqueeze(-1)
        view_dirs_encoded = [view_dirs]
        for l_dir in range(self.L_dir):
            view_dirs_encoded.append(torch.sin(2**l_dir*torch.pi*view_dirs))
            view_dirs_encoded.append(torch.cos(2**l_dir*torch.pi*view_dirs))
        
        view_dirs_encoded = torch.cat(view_dirs_encoded, dim=-1) ##(16*24)
        
        
        print(f'shape of view_dirs_encoded:{view_dirs_encoded.shape}')
        
        outputs_mlp_block1 = self.mlp_block_1(rays_samples_encoded)
        print(f'outputs_mlp_block1:{outputs_mlp_block1.shape}')
        outputs_mlp_block2 = self.mlp_block_2(view_dirs_encoded) ## outputs1 + outputs 2
        print(f'outputs_mlp_block2:{outputs_mlp_block2.shape}')
        # Sum the outputs 
        outputs_combined = outputs_mlp_block1+outputs_mlp_block2
        print(f'outputs_combined:{outputs_combined.shape}')
        # Now Concatenate with positional and directional embeddings
        outputs_mlp_block3 = self.mlp_block_3(outputs_combined)
        print(f'outputs_mlp_block3:{outputs_mlp_block3.shape}')
        
        
        

In [11]:
vanilla_skip_nerf_12_model1 = vanilla_skip_nerf_12()

In [12]:
vanilla_skip_nerf_12_model1(ray_dirs,ds_batch)

shape of rays_samples_encoded:torch.Size([16834, 63])
shape of view_dirs_encoded:torch.Size([16834, 27])
outputs_mlp_block1:torch.Size([16834, 256])
outputs_mlp_block2:torch.Size([16834, 256])
outputs_combined:torch.Size([16834, 256])
outputs_mlp_block3:torch.Size([16834, 256])


In [29]:
## In this architecture we have added output of a 4 block MLP of the positional and directional encodings and have just concatenated the directional encodings at the third block and have kept everything same
class vanilla_skip_nerf_12(nn.Module):
    def __init__(self):
        super().__init__()
        self.L_pos = 10
        self.L_dir = 4
        pos_enc_features = 3+3*2*self.L_pos 
        dir_enc_features = 3+3*2*self.L_dir 
        in_pos_features1 = pos_enc_features
        in_dir_features2 = dir_enc_features
        num_neurons = 256
        
        # In one mlp_block, we would feed the pos
        # In the other we would  feed the dir enc
        self.mlp_block_1 = nn.Sequential(
            nn.Linear(in_pos_features1,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        self.mlp_block_2 = nn.Sequential(
            nn.Linear(in_dir_features2, 256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        in_features_3 = num_neurons+dir_enc_features
        self.mlp_block_3 = nn.Sequential(
            nn.Linear(in_features_3,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        self.sigma_layer = nn.Linear(num_neurons, num_neurons+1) ## In next version just change the number of neurons
        self.pre_final_layer = nn.Sequential(nn.Linear(num_neurons, num_neurons//2),
                                            nn.ReLU())
        self.final_layer= nn.Sequential(nn.Linear(num_neurons//2,3,nn.Sigmoid()))
        
    def forward(self, rays_samples,view_dirs):
        rays_samples_encoded = [rays_samples]
        for l_pos in range(self.L_pos):
            rays_samples_encoded.append(torch.sin(2**l_pos*torch.pi*rays_samples))
            rays_samples_encoded.append(torch.cos(2**l_pos*torch.pi*rays_samples))
        
        rays_samples_encoded = torch.cat(rays_samples_encoded, dim=-1) ##(16K *60)
        print(f'shape of rays_samples_encoded:{rays_samples_encoded.shape}')
        
        
        view_dirs = view_dirs/view_dirs.norm(p=2, dim=-1).unsqueeze(-1)
        view_dirs_encoded = [view_dirs]
        for l_dir in range(self.L_dir):
            view_dirs_encoded.append(torch.sin(2**l_dir*torch.pi*view_dirs))
            view_dirs_encoded.append(torch.cos(2**l_dir*torch.pi*view_dirs))
        
        view_dirs_encoded = torch.cat(view_dirs_encoded, dim=-1) ##(16*24)
        
        
        print(f'shape of view_dirs_encoded:{view_dirs_encoded.shape}')
        
        outputs_mlp_block1 = self.mlp_block_1(rays_samples_encoded)
        print(f'outputs_mlp_block1:{outputs_mlp_block1.shape}')
        outputs_mlp_block2 = self.mlp_block_2(view_dirs_encoded) ## outputs1 + outputs 2
        print(f'outputs_mlp_block2:{outputs_mlp_block2.shape}')
        # Sum the outputs 
        outputs_combined = outputs_mlp_block1+outputs_mlp_block2
        print(f'outputs_combined:{outputs_combined.shape}')
        output_concat = torch.cat([outputs_combined,view_dirs_encoded],dim=-1) ## We have just concatenated the directional encodings
        print(f'shape of output concat: {output_concat.shape}') 

        # Now Concatenate with positional and directional embeddings
        outputs_mlp_block3 = self.mlp_block_3(output_concat)
        print(f'outputs_mlp_block3:{outputs_mlp_block3.shape}')
        ## At this point we are again concatenating just the directional stuff
        outputs = self.sigma_layer(outputs_mlp_block3)
        print(f'After passing sigma layer: {outputs_mlp_block3.shape}')
        sigma_is = torch.relu(outputs[:,0])
        print(f'sigma is: {sigma_is.shape}')
        outputs1 = self.pre_final_layer(outputs[:,1:])
        print(f'shape of output after passing pre_final_layer: {outputs1.shape}')
        c_is = self.final_layer(outputs1)
        print(f'shape of c_is:{c_is.shape}')
        return {"c_is":c_is, "sigma_is": sigma_is}
        
        
        
        

In [30]:
vanilla_skip_nerf_12_model2 = vanilla_skip_nerf_12()

In [31]:
vanilla_skip_nerf_12_model2(ray_dirs,ds_batch)

shape of rays_samples_encoded:torch.Size([16834, 63])
shape of view_dirs_encoded:torch.Size([16834, 27])
outputs_mlp_block1:torch.Size([16834, 256])
outputs_mlp_block2:torch.Size([16834, 256])
outputs_combined:torch.Size([16834, 256])
shape of output concat: torch.Size([16834, 283])
outputs_mlp_block3:torch.Size([16834, 256])
After passing sigma layer: torch.Size([16834, 256])
sigma is: torch.Size([16834])
shape of output after passing pre_final_layer: torch.Size([16834, 128])
shape of c_is:torch.Size([16834, 3])


{'c_is': tensor([[-0.0149,  0.0030, -0.0112],
         [-0.0154,  0.0034, -0.0118],
         [-0.0152,  0.0036, -0.0111],
         ...,
         [-0.0147,  0.0037, -0.0114],
         [-0.0146,  0.0037, -0.0111],
         [-0.0154,  0.0038, -0.0124]], grad_fn=<AddmmBackward0>),
 'sigma_is': tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<ReluBackward0>)}

Looks Like this nerf architecture do not work well, it starts with a huge lose compared to the basic model

In [44]:
class vanilla_skip_nerf_12_v1(nn.Module):
    def __init__(self):
        super().__init__()
        self.L_pos = 10
        self.L_dir = 4
        pos_enc_features = 3+3*2*self.L_pos 
        dir_enc_features = 3+3*2*self.L_dir 
        in_pos_features1 = pos_enc_features
        
        num_neurons = 256
        in_dir_features2 = num_neurons+dir_enc_features
       
        self.mlp_block_1 = nn.Sequential(
            nn.Linear(in_pos_features1,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        self.mlp_block_2 = nn.Sequential(
            nn.Linear(in_dir_features2, 256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        in_features_3 = num_neurons
        self.mlp_block_3 = nn.Sequential(
            nn.Linear(in_features_3,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU())
        self.sigma_layer = nn.Linear(num_neurons, num_neurons+1) ## In next version just change the number of neurons
        self.pre_final_layer = nn.Sequential(nn.Linear(num_neurons, num_neurons//2),
                                            nn.ReLU())
        self.final_layer= nn.Sequential(nn.Linear(num_neurons//2,3,nn.Sigmoid()))
        
    def forward(self, rays_samples,view_dirs):
        rays_samples_encoded = [rays_samples]
        for l_pos in range(self.L_pos):
            rays_samples_encoded.append(torch.sin(2**l_pos*torch.pi*rays_samples))
            rays_samples_encoded.append(torch.cos(2**l_pos*torch.pi*rays_samples))
        
        rays_samples_encoded = torch.cat(rays_samples_encoded, dim=-1) ##(16K *60)
        print(f'shape of rays_samples_encoded:{rays_samples_encoded.shape}')
        
        
        view_dirs = view_dirs/view_dirs.norm(p=2, dim=-1).unsqueeze(-1)
        view_dirs_encoded = [view_dirs]
        for l_dir in range(self.L_dir):
            view_dirs_encoded.append(torch.sin(2**l_dir*torch.pi*view_dirs))
            view_dirs_encoded.append(torch.cos(2**l_dir*torch.pi*view_dirs))
        
        view_dirs_encoded = torch.cat(view_dirs_encoded, dim=-1) ##(16*24)
        
        
        print(f'shape of view_dirs_encoded:{view_dirs_encoded.shape}')
        
        outputs_mlp_block1 = self.mlp_block_1(rays_samples_encoded)
        print(f'outputs_mlp_block1:{outputs_mlp_block1.shape}')
        outputs_mlp_block2 = self.mlp_block_2(torch.cat([view_dirs_encoded,outputs_mlp_block1], dim=-1)) ##concatenating directional inputs to outputs of previous MLP
        print(f'outputs_mlp_block2:{outputs_mlp_block2.shape}')
        # Sum the outputs 
        outputs_combined = outputs_mlp_block1+outputs_mlp_block2 ## Here goes the skip connections 
        print(f'outputs_combined:{outputs_combined.shape}')
#         output_concat = torch.cat([outputs_combined,view_dirs_encoded],dim=-1) ## We have just concatenated the directional encodings
#         print(f'shape of output concat: {output_concat.shape}') 

        # Now Concatenate with positional and directional embeddings
        outputs_mlp_block3 = self.mlp_block_3(outputs_combined)
        print(f'outputs_mlp_block3:{outputs_mlp_block3.shape}')
        ## At this point we are again concatenating just the directional stuff
        outputs = self.sigma_layer(outputs_mlp_block3)
        print(f'After passing sigma layer: {outputs_mlp_block3.shape}')
        sigma_is = torch.relu(outputs[:,0])
        print(f'sigma is: {sigma_is.shape}')
        outputs1 = self.pre_final_layer(outputs[:,1:])
        print(f'shape of output after passing pre_final_layer: {outputs1.shape}')
        c_is = self.final_layer(outputs1)
        print(f'shape of c_is:{c_is.shape}')
        return {"c_is":c_is, "sigma_is": sigma_is}
        
        
        
        

In [45]:
vanilla_skip_nerf_12_model3 = vanilla_skip_nerf_12_v1()

In [46]:
vanilla_skip_nerf_12_model3(ray_dirs,ds_batch)

shape of rays_samples_encoded:torch.Size([16834, 63])
shape of view_dirs_encoded:torch.Size([16834, 27])
outputs_mlp_block1:torch.Size([16834, 256])
outputs_mlp_block2:torch.Size([16834, 256])
outputs_combined:torch.Size([16834, 256])
outputs_mlp_block3:torch.Size([16834, 256])
After passing sigma layer: torch.Size([16834, 256])
sigma is: torch.Size([16834])
shape of output after passing pre_final_layer: torch.Size([16834, 128])
shape of c_is:torch.Size([16834, 3])


{'c_is': tensor([[-0.0538,  0.0169,  0.0172],
         [-0.0538,  0.0167,  0.0172],
         [-0.0538,  0.0167,  0.0171],
         ...,
         [-0.0538,  0.0167,  0.0172],
         [-0.0539,  0.0166,  0.0172],
         [-0.0537,  0.0166,  0.0172]], grad_fn=<AddmmBackward0>),
 'sigma_is': tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<ReluBackward0>)}